# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from preprocessing import *
from sql_queries import *

# ETL PART 1
##### Creating list of filepaths to process original event csv data files
##### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
processing_csv(filepath)


/home/workspace


In [3]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [4]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [5]:
try:
    session.execute(keyspace_create)
    
except Exception as e:
    print(e)

#### Set Keyspace

In [6]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4





In [7]:
session.execute(session_q_drop)
create_q = session_q_create

try:
    session.execute(create_q)
except Exception as e:
    print(e)
   

In [8]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        session.execute(session_q_insert, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [9]:
try:
    rows = session.execute(select_q_session)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [10]:
session.execute(user_q_drop)
try:
    session.execute(user_q_create)
except Exception as e:
    print(e)                 

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        session.execute(user_q_insert, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [12]:
try:
    rows = session.execute(select_q_user)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [13]:
session.execute(song_q_drop)
try:
    session.execute(song_q_create)
except Exception as e:
    print(e)                 

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        session.execute(song_q_insert, (line[9], int(line[10]), line[1], line[4]))

In [15]:
try:
    rows = session.execute(select_q_song)
except Exception as e:
    print(e)

for row in rows:
    print (row.userid, row.firstname, row.lastname)

29 Jacqueline Lynch
80 Tegan Levine
95 Sara Johnson


### Drop the tables before closing out the sessions

In [16]:
try:
    rows = session.execute(session_q_drop)
except Exception as e:
    print(e)
    
try:
    rows = session.execute(user_q_drop)
except Exception as e:
    print(e)
    
try:
    rows = session.execute(song_q_drop)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [17]:
session.shutdown()
cluster.shutdown()